# CVC Data Summaries (with simple method hydrology)

## Setup the basic working environment

In [ ]:
%matplotlib inline

import os
import sys
import datetime
import warnings

import numpy as np
import matplotlib.pyplot as plt
import pandas
import seaborn
seaborn.set(style='ticks', context='paper')

import wqio
import pybmpdb
import pynsqd

import pycvc

min_precip = 1.9999
palette = seaborn.color_palette('deep', n_colors=6)
pybmpdb.setMPLStyle()
POCs = [p['cvcname'] for p in filter(lambda p: p['include'], pycvc.info.POC_dicts)]

if wqio.testing.checkdep_tex() is None:
    tex_msg = ("LaTeX not found on system path. You will "
               "not be able to compile ISRs to PDF files")
    warnings.warn(tex_msg, UserWarning)
    
warning_filter = "ignore" 
warnings.simplefilter(warning_filter)

## Load External Data (this takes a while)

In [ ]:
bmpdb = pycvc.external.bmpdb(palette[3], 'D')
nsqdata = pycvc.external.nsqd(palette[2], 'd')

## Load CVC Database

In [ ]:
cvcdbfile = "C:/users/phobson/Desktop/cvc.accdb"
cvcdb = pycvc.Database(cvcdbfile, nsqdata, bmpdb, testing=False)

## Define the site object for the reference site and compute its median values ("influent" to other sites)

In [ ]:
LV1 = pycvc.Site(db=cvcdb, siteid='LV-1', raingauge='LV-1', tocentry='Lakeview Control', 
                 isreference=True,  minprecip=min_precip, color=palette[1], marker='s')

### Define the reference sites runoff function and assign it to the object

In [ ]:
def LV1_runoff(row):
    return LV1.drainagearea.simple_method(row['total_precip_depth'], volume_conversion=0.001)

LV1.runoff_fxn = LV1_runoff

## Define "influent" medians for each non-reference site

In [ ]:
def rename_influent_cols(col):
    if col.lower() in ['parameter', 'units', 'season']:
        newcol = col.lower()
    else:
        newcol = 'influent {}'.format(col.lower())
        
    return newcol.replace(' nsqd ', ' ').replace(' effluent ', ' ')

### Lakeview BMP sites get their "influent" data from LV-1

In [ ]:
LV_Influent = (
    LV1.medians("concentration", timegroup='season')
       .rename(columns={'effluent stat': 'median'})
       .rename(columns=rename_influent_cols)
)

LV1.influentmedians = LV_Influent
LV_Influent.head()

### Elm Drive's "influent" data come from NSQD

In [ ]:
ED_Influent = (
    cvcdb.nsqdata
         .seasonal_medians
         .rename(columns=rename_influent_cols)
)
ED_Influent.head()

### Remaining site objects

In [ ]:
ED1 = pycvc.Site(db=cvcdb, siteid='ED-1', raingauge='ED-1',
                 tocentry='Elm Drive', influentmedians=ED_Influent, 
                 minprecip=min_precip, isreference=False,
                 color=palette[0], marker='o')

LV2 = pycvc.Site(db=cvcdb, siteid='LV-2', raingauge='LV-1',
                 tocentry='Lakeview Grass Swale', influentmedians=LV_Influent, 
                 minprecip=min_precip, isreference=False,
                 color=palette[4], marker='^')

LV4 = pycvc.Site(db=cvcdb, siteid='LV-4', raingauge='LV-1',
                 tocentry=r'Lakeview Bioswale 1$^{\mathrm{st}}$ South Side', 
                 influentmedians=LV_Influent, 
                 minprecip=min_precip, isreference=False,
                 color=palette[5], marker='v')

### Define runoff, inflow equations and assign to object

#### Elm Drive

In [ ]:
def ED1_runoff(row):
    return ED1.drainagearea.simple_method(row['total_precip_depth'], volume_conversion=0.001)

def ED1_inflow(row):
    return ED1_runoff(row)

ED1.runoff_fxn = ED1_runoff
ED1.inflow_fxn = ED1_inflow

#### Lakeview 2

In [ ]:
def LV2_runoff(row):
    return LV2.drainagearea.simple_method(row['total_precip_depth'], volume_conversion=0.001)

def LV2_inflow(row):
    return LV2_runoff(row)

LV2.runoff_fxn = LV2_runoff
LV2.inflow_fxn = LV2_inflow

#### Lakeview 4

In [ ]:
def LV4_runoff(row):
    return LV4.drainagearea.simple_method(row['total_precip_depth'], volume_conversion=0.001)

def LV4_inflow(row):
    return LV4_runoff(row)

LV4.runoff_fxn = LV4_runoff
LV4.inflow_fxn = LV4_inflow

### Fix ED-1 storm that had two composite samples

In [ ]:
ED1.hydrodata.data.loc['2012-08-10 23:50:00':'2012-08-11 05:20', 'storm'] = 0
ED1.hydrodata.data.loc['2012-08-11 05:30':, 'storm'] += 1

### Use spreadsheet-modeled outflow for the July 8, 2013 event at ED-1

In [ ]:
# volume from the spreadsheet model
modeled_inflow_Liters = 430603
modeled_outflow_Liters = 250965

# select the big storm
storm_date = datetime.date(2013, 7, 8)
bigstorm = ED1.storm_info.loc[ED1.storm_info.start_date.dt.date == storm_date].iloc[0]

# overwrite values in the storm_info dataframe
ED1.storm_info.loc[bigstorm.name, 'inflow_m3'] = modeled_inflow_Liters / pycvc.info.LITERS_PER_CUBICMETER
ED1.storm_info.loc[bigstorm.name, 'outflow_m3'] = modeled_outflow_Liters / pycvc.info.LITERS_PER_CUBICMETER

# modify the volumes in the individual storm objects
ED1.storms[bigstorm.storm_number].total_inflow_volume = modeled_inflow_Liters / pycvc.info.LITERS_PER_CUBICMETER
ED1.storms[bigstorm.storm_number].total_outflow_volume = modeled_outflow_Liters / pycvc.info.LITERS_PER_CUBICMETER

## High-level summaries

### Hydrologic Summary
For LV-1 and LV-2, event durations are winsorized to replace outliers beyond the 97.5 percentile.

For more information, see:

  1. [scipy.stats.mstats.winsorize](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html)
  2. [wqio.utils.winsorize_dataframe](https://github.com/Geosyntec/wqio/blob/6a056bee34f9c3ed8e300c3d6db1130a6f9ce980/wqio/utils/misc.py#L1548)

In [ ]:
winsor_limits = {
    'ED-1': None,
    'LV-1': (0.000, 0.025),
    'LV-2': (0.000, 0.025),
    'LV-4': None,
}

with pandas.ExcelWriter("output/xlsx/CVCHydro_StormInfo.xlsx") as stormfile,\
     pandas.ExcelWriter("output/xlsx/CVCHydro_StormStats.xlsx") as allstats,\
     pandas.ExcelWriter("output/xlsx/CVCHydro_StormStats_by_Year.xlsx") as yearstats,\
     pandas.ExcelWriter("output/xlsx/CVCHydro_StormStats_by_Season.xlsx") as seasonstats,\
     pandas.ExcelWriter("output/xlsx/CVCHydro_StormStats_by_GroupSeason.xlsx") as groupstats:

    for site in [ED1, LV1, LV2, LV4]:
        stat_options = {'duration_hours': winsor_limits[site.siteid]}
        site.storm_info.to_excel(stormfile, sheet_name=site.siteid)
        site.storm_stats(timegroup=None, **stat_options).to_excel(allstats, sheet_name=site.siteid)
        site.storm_stats(timegroup='year', **stat_options).to_excel(yearstats, sheet_name=site.siteid)
        site.storm_stats(timegroup='season', **stat_options).to_excel(seasonstats, sheet_name=site.siteid)
        site.storm_stats(timegroup='grouped_season', **stat_options).to_excel(groupstats, sheet_name=site.siteid)

### Hydrologic Pairplots 
Expected failures due to lack of data:
  1. LV-2, outflow
  1. LV-4, grouped_season

In [ ]:
for site in [ED1, LV2, LV4]:
    for by in ['year', 'outflow', 'season', 'grouped_season']:
        try:               
            site.hydro_pairplot(by=by)
        except:              
            print('failed on {}, {}'.format(site, by))

### Prevalence Tables

In [ ]:
with pandas.ExcelWriter('output/xlsx/CVCWQ_DataInventory.xlsx') as prev_tables:
    for site in [ED1, LV1, LV2, LV4]:
        stype = 'composite'
        site.prevalence_table()[stype].to_excel(prev_tables, sheet_name='{}'.format(site.siteid))

### Concentrations Stats

In [ ]:
with pandas.ExcelWriter('output/xlsx/CVCWQ_ConcStats.xlsx') as allconc, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_ConcStats_by_Year.xlsx') as yearconc, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_ConcStats_by_Season.xlsx') as seasonconc, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_ConcStats_by_GroupedSeason.xlsx') as groupconc:
    for site in [ED1, LV1, LV2, LV4]:
        sheetopts = dict(sheet_name=site.siteid, na_rep='--')
        summaryopts = dict(rescol='concentration', sampletype='composite')
        site.wq_summary(timegroup=None, **summaryopts).to_excel(allconc, **sheetopts)
        site.wq_summary(timegroup='year', **summaryopts).to_excel(yearconc, **sheetopts)
        site.wq_summary(timegroup='season', **summaryopts).to_excel(seasonconc, **sheetopts)
        site.wq_summary(timegroup='grouped_season', **summaryopts).to_excel(groupconc, **sheetopts)

### Total Loads Summary

In [ ]:
with pandas.ExcelWriter('output/xlsx/CVCWQ_SampledLoads.xlsx') as allloads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_SampledLoads_by_Season.xlsx') as seasonloads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_SampledLoads_by_GroupedSeason.xlsx') as grouploads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_SampledLoads_by_Year.xlsx') as yearloads:
    for site in [ED1, LV1, LV2, LV4]:
        load_opts = dict(sampletype='composite', NAval=0)
        sheetopts = dict(sheet_name=site.siteid, na_rep='--')
        site.sampled_loads(**load_opts).to_excel(allloads, **sheetopts)
        site.sampled_loads(timegroup='season', **load_opts).to_excel(seasonloads, **sheetopts)
        site.sampled_loads(timegroup='grouped_season', **load_opts).to_excel(grouploads, **sheetopts)
        site.sampled_loads(timegroup='year', **load_opts).to_excel(yearloads, **sheetopts)


### Load Stats

In [ ]:
with pandas.ExcelWriter('output/xlsx/CVCWQ_LoadStats.xlsx') as allloads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_LoadStats_by_Year.xlsx') as yearloads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_LoadStats_by_Season.xlsx') as seasonloads, \
     pandas.ExcelWriter('output/xlsx/CVCWQ_LoadStats_by_GroupedSeason.xlsx') as grouploads:
    for site in [ED1, LV1, LV2, LV4]:
        sheetopts = dict(sheet_name=site.siteid, na_rep='--')
        summaryopts = dict(rescol='load_outflow', sampletype='composite')
        site.wq_summary(timegroup=None, **summaryopts).to_excel(allloads, **sheetopts)
        site.wq_summary(timegroup='year', **summaryopts).to_excel(yearloads, **sheetopts)
        site.wq_summary(timegroup='season', **summaryopts).to_excel(seasonloads, **sheetopts)
        site.wq_summary(timegroup='grouped_season', **summaryopts).to_excel(grouploads, **sheetopts)

### Tidy Data

In [ ]:
with pandas.ExcelWriter('output/xlsx/CVCWQ_TidyData.xlsx') as tidyfile:
    for site in [ED1, LV1, LV2, LV4]:
        site.tidy_data.to_excel(tidyfile, sheet_name=site.siteid, na_rep='--')

## Analysis

In [ ]:
seaborn.set(style='ticks', context='paper')
pybmpdb.setMPLStyle()

### Individual Storm Reports
(requires $\LaTeX$)

In [ ]:
for site in [ED1, LV1, LV2, LV4]:
    print('\n----Compiling ISR for {0}----'.format(site.siteid))
    site.allISRs('composite', version='draft')

### Precip-outflow scatter plots

In [ ]:
for site in [ED1, LV1, LV2, LV4]:
    print('\n----Summarizing {0}----'.format(site.siteid))
    
    site.hydro_jointplot(
        xcol='total_precip_depth', 
        ycol='outflow_mm', 
        conditions="outflow_mm > 0", 
        one2one=True
    )

    site.hydro_jointplot(
        xcol='antecedent_days', 
        ycol='outflow_mm', 
        conditions="outflow_mm > 0", 
        one2one=False
    )

    site.hydro_jointplot(
        xcol='total_precip_depth', 
        ycol='antecedent_days', 
        conditions="outflow_mm == 0", 
        one2one=False
    )
    
    site.hydro_jointplot(
        xcol='peak_precip_intensity', 
        ycol='peak_outflow', 
        conditions=None, 
        one2one=False
    )
    
    plt.close('all')

## WQ Comparison

### Lists of sites to compare

In [ ]:
site_lists = [
    [ED1],
    [LV1, LV2, LV4],
]

### Individual Figures

In [ ]:
for sl in site_lists:
    print('\n----Comparing {}----'.format(', '.join([s.siteid for s in sl])))
    for poc in POCs:
        print('  ' + poc)
        
        wqcomp = pycvc.summary.WQComparison(sl, 'composite', poc, nsqdata, bmpdb)
        
        wqcomp.seasonalBoxplots(load=False, finalOutput=True)
        wqcomp.seasonalBoxplots(load=True, finalOutput=True)
        
        wqcomp.landuseBoxplots(finalOutput=True)
        wqcomp.bmpCategoryBoxplots(finalOutput=True)
        
        wqcomp.parameterStatPlot(finalOutput=True)
        wqcomp.parameterStatPlot(load=True, finalOutput=True)
        
        wqcomp.parameterTimeSeries(finalOutput=True)  
        wqcomp.parameterTimeSeries(load=True, finalOutput=True)  

        plt.close('all')

### Megafigures

In [ ]:
for sl in site_lists:
    print('\n----Megafigs with {}----'.format(', '.join([s.siteid for s in sl])))
    
    # construct the megafigures
    mf1 = pycvc.summary.WQMegaFigure(sl, 'composite', POCs[:6], 1, nsqdata, bmpdb)
    mf2 = pycvc.summary.WQMegaFigure(sl, 'composite', POCs[6:], 2, nsqdata, bmpdb)
    for n, mf in enumerate([mf1, mf2]):
        print('\tTime Series {0}'.format(n+1))
        mf.timeseriesFigure(load=False)
        mf.timeseriesFigure(load=True)

        print('\tStat plots {0}'.format(n+1))
        mf.statplotFigure(load=False)
        mf.statplotFigure(load=True)

        print('\tBMPDB Boxplots {0}'.format(n+1))
        mf.bmpCategoryBoxplotFigure()

        print('\tNSQD Boxplots {0}'.format(n+1))
        mf.landuseBoxplotFigure()

        print('\tSeasonal Boxplots {0}'.format(n+1))
        mf.seasonalBoxplotFigure(load=False)
        mf.seasonalBoxplotFigure(load=True)
     
    plt.close('all')

### Unsampled loading estimates
*Warning*: Site objects (e.g., `ED1`) have hidden `_unsampled_loading_estimates` methods that return load estimates of unsampled storms using the estimated median influent concentrations and median effluent concentrations. However, it is highly recommended that you aggregate the data and don't draw conclusions about individual storms.

The cell below aggregates the data for each parameter, season, and whether the storms produced outflow. The results (sums) are then saved to an Excel file, one tab for each site.

In [ ]:
with pandas.ExcelWriter("output/xlsx/CVCWQ_UnsampledLoadEstimates.xlsx") as unsampled_all, \
     pandas.ExcelWriter("output/xlsx/CVCWQ_UnsampledLoadEstimates_by_Season.xlsx") as unsampled_seasons, \
     pandas.ExcelWriter("output/xlsx/CVCWQ_UnsampledLoadEstimates_by_GroupedSeason.xlsx") as unsampled_groups, \
     pandas.ExcelWriter("output/xlsx/CVCWQ_UnsampledLoadEstimates_by_Year.xlsx") as unsampled_years:
    for site in [ED1, LV1, LV2, LV4]:
        site._unsampled_load_estimates(NAval=0).to_excel(unsampled_all, sheet_name=site.siteid)
        site._unsampled_load_estimates(NAval=0, timegroup='season').to_excel(unsampled_seasons, sheet_name=site.siteid)
        site._unsampled_load_estimates(NAval=0, timegroup='grouped_season').to_excel(unsampled_groups, sheet_name=site.siteid)
        site._unsampled_load_estimates(NAval=0, timegroup='year').to_excel(unsampled_years, sheet_name=site.siteid)